In [1]:
import numpy as np
import pandas as pd
import rpy2.robjects as ro
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri
from rpy2.robjects.conversion import localconverter
import os

In [2]:
# paths
raw_path = '/home/usuario/Documents/Barcelona_Yr1/GraphicalModels_NetworkData/LiLicode/COVID_DATA/'
flight_data_path_root = '/home/usuario/Documents/Barcelona_Yr1/GraphicalModels_NetworkData/LiLicode/COVID_DATA/flight_compatible_data/YabraFlightData/'
save_path_root = '/home/usuario/Documents/Barcelona_Yr1/GraphicalModels_NetworkData/LiLicode/COVID_DATA/flight_compatible_data/'

# Data preprocessing (missing merging FIPS with available FB data)

In [3]:
nodes = pd.read_csv(raw_path+'nodes118.csv', index_col='Unnamed: 0')

In [4]:
area = pd.read_excel(raw_path+'US County Area.xlsx')
area['NAME'] = area.NAME.apply(lambda x: x.lower())

In [5]:
fips = pd.read_csv(raw_path+'FIPS code.csv')
fips['name'] = fips.name.apply(lambda x: x.lower())

In [6]:
pop = pd.read_excel(raw_path+'2019 Population2.xlsx')
pop['name'] = pop.name.apply(lambda x: x.lstrip('.').lower())

# split name into county name and state
pop[['countyname', 'State']] = pop['name'].str.split(',', 1, expand=True)
pop['State'] = pop.State.apply(lambda x: x.lstrip(' '))

In [7]:
pop = pop.drop(['Unnamed: 2','Unnamed: 3'], axis = 1)
pop.head()

,name,Population,countyname,State
0,"los angeles county, california",10039107,los angeles county,california
1,"cook county, illinois",5150233,cook county,illinois
2,"harris county, texas",4713325,harris county,texas
3,"maricopa county, arizona",4485414,maricopa county,arizona
4,"san diego county, california",3338330,san diego county,california


In [8]:
# merge population with FIPS by name
popcode = pop.merge(fips, left_on='name', right_on='name', how='left')

In [9]:
# manual fixing where merging failed
popcode[popcode.code.isna()]

,name,Population,countyname,State,code
394,"maui county, hawaii",167417,maui county,hawaii,NaN
2178,"o'brien county, iowa",13753,o'brien county,iowa,NaN
3143,"kalawao county, hawaii",86,kalawao county,hawaii,NaN


In [10]:
# 1: filling with "maui" from area (in fips maui is merged with kalawao under maui FIPS code)
popcode.loc[popcode['name'].str.contains('maui'), 'code'] = area[area['NAME'].str.contains('maui')]['GEOID'].values

In [11]:
# 2: filling with "obrien" from fips (without apostrophe)
popcode.loc[popcode['name'].str.contains('brien'), 'code'] = fips[fips['name'].str.contains('brien')]['code'].values

In [12]:
# 3: filling with "kalawao" from area (in fips maui is merged with kalawao under maui FIPS code)
popcode.loc[popcode['name'].str.contains('kalawao'), 'code'] = area[area['NAME'].str.contains('kalawao')]['GEOID'].values

In [13]:
popcode = popcode.reset_index(drop=True).copy()
popcode['code'] = popcode['code'].astype(int).copy()

In [14]:
# merge with longitude and latitude from area
df = popcode.merge(area, left_on='code', right_on='GEOID', how='left')

In [15]:
# long and lat missing for these three, dropping them
df[df.GEOID.isna()]

,name,Population,countyname,State,code,GEOID,COUNTYNS,BASENAME,NAME,AREALAND,AREAWATER,CSA,CBSA,CENTLAT,CENTLON,INTPTLAT,INTPTLON
1928,"ste. genevieve county, missouri",17894,ste. genevieve county,missouri,29193,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2003,"park county, montana",16606,park county,montana,30901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2484,"valdez-cordova census area, alaska",9202,valdez-cordova census area,alaska,2261,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
df = df.dropna(subset=['GEOID']).reset_index(drop=True).copy()
df.rename(columns={'code':'FIPS'}, inplace=True)
df.shape

(3141, 17)

# Load Data preprocessed in R

In [17]:
daily = pd.read_csv(raw_path+'myfinal_daily_temp_ALL.csv')

In [18]:
flights = pd.read_csv(flight_data_path_root+'county_flows_people.csv')## includes caapcity
#flights = pd.read_csv(flight_data_path_root+'county_flows_flights.csv')## doesn't include capacity 

In [19]:
fips_to_id = daily[['ID', 'FIPS']].drop_duplicates().reset_index(drop=True).copy()

In [20]:
fips_to_id_dict = {k:v for k,v in zip(fips_to_id['FIPS'].values, fips_to_id['ID'].values)}

In [21]:
# sort of manual merge as I need to match 2 columns (county_origin, county_destination) to one column (FIPS)
fips_flights = np.unique(list(flights['county_origin'].unique()) + list(flights['county_destination'].unique()))
len(fips_flights)

3144

In [22]:
# fips in flights but not in daily
todrop_flights = list(set(fips_flights) - set(fips_to_id['FIPS'].values))

In [23]:
# fips in daily but not in flights (connecticut)
todrop_daily = list(set(fips_to_id['FIPS'].values) - set(fips_flights))

In [24]:
todrop_daily_IDs = [fips_to_id_dict[t] for t in todrop_daily]

In [25]:
print(flights.shape)
flights = flights[~flights.county_origin.isin(todrop_flights)].copy()
print(flights.shape)
flights = flights[~flights.county_destination.isin(todrop_flights)].copy()
flights

(9881592, 3)
(9834447, 3)


,county_origin,county_destination,flow
0,1001,1003,0.000000
1,1001,1005,0.000000
2,1001,1007,0.152283
3,1001,1009,0.403788
4,1001,1011,0.000000
...,...,...,...
9881587,56045,56035,0.000000
9881588,56045,56037,0.000000
9881589,56045,56039,0.000000
9881590,56045,56041,0.000000


In [26]:
print(fips_to_id.shape)
fips_to_id = fips_to_id[~fips_to_id.FIPS.isin(todrop_daily)].copy()
print(fips_to_id.shape)

(3137, 2)
(3129, 2)


In [27]:
# merge with IDs

In [28]:
flights = flights.merge(fips_to_id, left_on='county_origin', right_on='FIPS', how='inner').copy()
flights.columns = ['county_origin', 'county_destination', 'flow', 'ID_origin', 'FIPS_origin']

In [29]:
flights = flights.merge(fips_to_id, left_on='county_destination', right_on='FIPS', how='inner').copy()
flights.columns = ['county_origin', 'county_destination', 'flow', 'ID_origin', 'FIPS_origin', 'ID_destination', 'FIPS_destination']


In [30]:
%%time
flights['ID_o-d'] = flights.apply(lambda x:  '-'.join([str(i)
                                   for i in sorted([x['ID_origin']] + [x['ID_destination']])]),
              axis=1)

CPU times: user 1min 22s, sys: 662 ms, total: 1min 23s
Wall time: 1min 23s


In [31]:
%%time
flights_undir = flights.groupby('ID_o-d')[['flow']].sum().reset_index().copy()

CPU times: user 9.48 s, sys: 160 ms, total: 9.64 s
Wall time: 9.63 s


In [32]:
%%time
flights_undir['ID_a'] = flights_undir.apply(lambda x: int(x['ID_o-d'][:x['ID_o-d'].find('.')]), axis=1)

CPU times: user 35.2 s, sys: 252 ms, total: 35.4 s
Wall time: 35.4 s


In [33]:
%%time
flights_undir['ID_b'] = flights_undir.apply(lambda x: int(x['ID_o-d'][x['ID_o-d'].find('-')+1:-2]), axis=1)

CPU times: user 36.2 s, sys: 272 ms, total: 36.5 s
Wall time: 36.5 s


In [34]:
(flights_undir['ID_a']==flights_undir['ID_b']).sum()

0

In [35]:
(flights_undir['ID_a']>flights_undir['ID_b']).sum()

0

In [36]:
flights_undir.drop(['ID_o-d'], axis=1, inplace=True)
flights_undir

,flow,ID_a,ID_b
0,39362.434013,1,10
1,178807.062313,1,100
2,0.000000,1,1000
3,0.000000,1,1001
4,0.000000,1,1002
...,...,...,...
4893751,0.000000,999,3139
4893752,0.000000,999,3140
4893753,0.000000,999,3141
4893754,0.000000,999,3142


In [37]:
%%time
flights_adj_mat = pd.crosstab(flights_undir['ID_a'], 
                              flights_undir['ID_b'], 
                              values=flights_undir['flow'], 
                              aggfunc='sum', dropna=False)


CPU times: user 1.19 s, sys: 272 ms, total: 1.46 s
Wall time: 1.46 s


In [38]:
flights_adj_mat.loc[3143] = len(flights_adj_mat.index)*[np.nan]

In [39]:
flights_adj_mat.loc[:,1] = len(flights_adj_mat.index)*[np.nan]

In [40]:
flights_adj_mat = flights_adj_mat[sorted(flights_adj_mat.columns)]

In [41]:
flights_adj_mat = (flights_adj_mat.fillna(0.0).T + flights_adj_mat.fillna(0.0)).copy()

In [42]:
flights_adj_mat['ID'] = [int(s) for s in flights_adj_mat.columns]

In [43]:
flights_adj_mat.reset_index(drop=True, inplace=True)

In [44]:
flights_adj_mat.to_csv(raw_path + f'flights_capacity_mat.csv')
#flights_adj_mat.to_csv(raw_path + f'flights_nocapacity_mat.csv')

In [45]:
# prune other networks

In [46]:
sci = pd.read_csv(raw_path+'SCI_index_ALL.csv')

In [47]:
sci['ID'] = [int(s) for s in sci.columns]
sci = sci[~sci['ID'].isin(todrop_daily_IDs)].copy()
sci = sci[[s for s in sci.columns if s not in [str(t) for t in todrop_daily_IDs]]].reset_index(drop=True).copy()

In [48]:
sci.to_csv(raw_path+'SCI_index_ALL_pruned.csv')

In [49]:
geodist = pd.read_csv(raw_path+'Geodist_ALL.csv')

In [50]:
geodist['ID'] = [int(s) for s in geodist.columns]
geodist = geodist[~geodist['ID'].isin(todrop_daily_IDs)].copy()
geodist = geodist[[s for s in geodist.columns if s not in [str(t) for t in todrop_daily_IDs]]].reset_index(drop=True).copy()

In [51]:
geodist.to_csv(raw_path+'Geodist_ALL_pruned.csv')

In [52]:
# prune df

In [53]:
df = pd.read_csv(raw_path+'confirmed_pop_ALL.csv')

In [54]:
df = df.merge(fips_to_id[['ID']], on='ID', how='right').reset_index(drop=True).copy()

In [55]:
df.to_csv(raw_path+'confirmed_pop_ALL_pruned.csv')

In [56]:
daily = daily.merge(fips_to_id[['ID']], on='ID', how='right').reset_index(drop=True).copy()

In [57]:
daily.to_csv(raw_path+'myfinal_daily_temp_ALL_pruned.csv')

In [58]:
# sanity checks 

set(sorted(df['ID'])) - set(sorted(geodist['ID']))

set()

In [59]:
set(sorted(df['ID'])) - set(sorted(sci['ID']))

set()

In [60]:
set(sorted(df['ID'])) - set(sorted(flights_adj_mat['ID']))

set()

In [61]:
set(np.arange(1,3144)) - set(sorted(df['ID']))

{54, 63, 68, 94, 257, 367, 395, 409, 442, 535, 1928, 2004, 2180, 2485}

In [62]:
set(np.arange(1,3144)) - set(sorted(geodist['ID']))

{54, 63, 68, 94, 257, 367, 395, 409, 442, 535, 1928, 2004, 2180, 2485}

In [63]:
set(np.arange(1,3144)) - set(sorted(sci['ID']))

{54, 63, 68, 94, 257, 367, 395, 409, 442, 535, 1928, 2004, 2180, 2485}

In [64]:
set(np.arange(1,3144)) - set(sorted(flights_adj_mat['ID']))

{54, 63, 68, 94, 257, 367, 395, 409, 442, 535, 1928, 2004, 2180, 2485}

# Clustering

## Split into small and big counties

In [65]:
pop_thresh = 500000

In [66]:
big_counties = df[df.Population>=pop_thresh].reset_index(drop=True).copy()
big_counties.shape

(136, 690)

In [67]:
small_counties = df[df.Population<pop_thresh].reset_index(drop=True).copy()
small_counties.shape

(2993, 690)

In [68]:
states = list(set(small_counties.State.values))
len(states)

49

In [69]:
small_counties.groupby('State').FIPS.count().sort_values()

State
delaware            2
hawaii              2
rhode island        4
massachusetts       7
new hampshire      10
new jersey         12
arizona            13
vermont            14
nevada             16
maine              16
maryland           19
wyoming            23
utah               27
alaska             28
new mexico         32
oregon             34
washington         35
california         42
idaho              44
south carolina     45
north dakota       53
new york           53
florida            55
west virginia      55
montana            56
colorado           59
pennsylvania       60
louisiana          64
south dakota       66
alabama            66
wisconsin          70
arkansas           75
oklahoma           75
michigan           79
mississippi        82
ohio               83
minnesota          85
indiana            91
nebraska           92
tennessee          93
illinois           97
north carolina     97
iowa               98
kansas            103
missouri          113
kent

## Perform clustering

In [70]:
# Runs in R; if to_point is False, it produces a distance matrix between all points; 
# If a point is provided, it produces a distance vector of that point from any other point;
# Distance measure used: geodesic
# From geodist docs: ""geodesic" denotes the very accurate geodesic methods given in Karney (2013) "Algorithms for geodesics" J Geod 87:43-55,"
# We can use these distances as alternatives: "haversine" "vincenty", "cheap"

def pygeodist(pd_df, to_point=False, point=None):
    """
    pd_df must have columns 'x' and 'y' such that 'x' is the lng coordinate
    and 'y' is the lat coordinate
    """
    geodist=importr('geodist')
    with localconverter(ro.default_converter + pandas2ri.converter):
        if to_point:
            return geodist.geodist(pd_df, point, measure = "haversine")
        else:
            return geodist.geodist(pd_df, measure = "haversine")
        

In [71]:
all_res = {s:{} for s in states}

In [72]:
cols = ['Population',  'FIPS', 'Lat', 'Long_', 'State', 'ID'] # 'name', 'countyname',

In [73]:
%%time
for s_ix, s in enumerate(states):
    # For current state s:
    print(s_ix, s)

    state_df = small_counties[small_counties.State==s][cols].reset_index(drop=True).copy()
    state_df.columns = ['Population', 'FIPS', 'y', 'x', 'State', 'ID']

    # save mapping of ID to FIPS code
    idx_to_fips = {i:code for i,code in enumerate(state_df['FIPS'].values)}
    all_res[s]['idx_to_fips'] = idx_to_fips

    # compute distance between all points
    dist = pygeodist(state_df)
    
    # init tree as dict of {ID:emptylist} for each county
    tree = {i:[] for i in list(state_df.index)}

    # store initial smallest population value
    smallest_pop_val = state_df.sort_values('Population').reset_index()['Population'][0]

    # stopping criterion: all new counties are bigger than pop_thresh or the merge of all counties doesn't exceed pop_thresh
    while ((smallest_pop_val < pop_thresh) and (len(state_df['Population'].unique())!=1)):
        # ID of smallest county
        smallest = state_df.sort_values('Population').index[0]

        # ID of closest county that is not equal to smallest county
        closest = np.argsort(dist[smallest])[np.where(np.sort(dist[smallest])!=0)[0][0]]

        # new population, lat and long for smallest and closest county
        new_pop = state_df.iloc[smallest]['Population'] + state_df.iloc[closest]['Population'] 
        new_lat_y = (state_df.iloc[smallest]['y'] + state_df.iloc[closest]['y'])/2
        new_long_x = (state_df.iloc[smallest]['x'] + state_df.iloc[closest]['x'])/2

        # store group of closest counties and delete key where closest appears
        try:
            all_closest = [closest] + tree[closest]
            tree.pop(closest)
        except:
            for k,v in tree.copy().items():
                if closest in v:
                    all_closest = [k] + tree[k]
                    tree.pop(k)

        # update key where smallest appears adding all the closest counties
        try:
            tree[smallest] = tree[smallest] + all_closest
            list_to_update = [smallest] + tree[smallest]
        except:
            for k,v in tree.items():
                if smallest in v:
                    tree[k] = tree[k] + all_closest
                    list_to_update = [k] + tree[k]

        # update each smallest and closest county to new pop, lat, long
        for county in list_to_update:
            state_df.loc[state_df.index==county, 'Population'] = new_pop
            state_df.loc[state_df.index==county, 'y'] = new_lat_y
            state_df.loc[state_df.index==county, 'x'] = new_long_x

        # update dist for all counties with recalculated lat and long for list_to_update
        dist = pygeodist(state_df)

        # update smallest population value
        smallest_pop_val = state_df.sort_values('Population').reset_index()['Population'][0]

    # store tree and df for current state
    all_res[s]['tree'] = tree
    all_res[s]['state_df'] = state_df

0 new york
1 louisiana
2 south carolina
3 alaska
4 massachusetts
5 texas
6 kentucky
7 iowa
8 california
9 vermont
10 oklahoma
11 minnesota
12 rhode island
13 alabama
14 maine
15 new mexico
16 montana
17 new jersey
18 south dakota
19 florida
20 virginia
21 pennsylvania
22 michigan
23 wisconsin
24 indiana
25 mississippi
26 illinois
27 arizona
28 arkansas
29 utah
30 maryland
31 oregon
32 nebraska
33 hawaii
34 north carolina
35 ohio
36 delaware
37 north dakota
38 washington
39 new hampshire
40 georgia
41 idaho
42 kansas
43 wyoming
44 tennessee
45 missouri
46 nevada
47 west virginia
48 colorado
CPU times: user 6min 11s, sys: 20.4 s, total: 6min 31s
Wall time: 6min 32s


In [74]:
total_no_small_counties = sum([len(all_res[s]['tree']) for s in states])
total_no_small_counties

196

In [75]:
total_no_counties = big_counties.shape[0] + total_no_small_counties
total_no_counties

332

In [76]:
save_path = save_path_root + f'meta_flights_{total_no_counties}/'
if not os.path.exists(save_path):
    os.makedirs(save_path)

In [77]:
cols

['Population', 'FIPS', 'Lat', 'Long_', 'State', 'ID']

In [78]:
big_counties = big_counties[cols].copy()
big_counties.columns = ['Population', 'FIPS', 'y', 'x', 'State', 'ID']

In [79]:
# long_df has rows equal to original number of counties, their FIPS, but new population, long and lat
# We can use this to merge with other relevant info
long_df = pd.concat([all_res[s]['state_df'] for s in states]+[big_counties])
long_df = long_df.sort_values(['State', 'Population']).reset_index(drop=True).copy()
long_df.columns = cols
long_df.insert(loc=0, column='ID_metacounty', 
                   value=long_df.set_index(['Population','Lat', 'Long_']).index.factorize()[0])
long_df

,ID_metacounty,Population,FIPS,Lat,Long_,State,ID
0,0,619018,1089,34.444911,-85.998297,alabama,190
1,0,619018,1095,34.444911,-85.998297,alabama,622
2,0,619018,1049,34.444911,-85.998297,alabama,760
3,0,619018,1071,34.444911,-85.998297,alabama,965
4,0,619018,1019,34.444911,-85.998297,alabama,1559
...,...,...,...,...,...,...,...
3124,331,578759,56043,42.882320,-107.620012,wyoming,2612
3125,331,578759,56011,42.882320,-107.620012,wyoming,2625
3126,331,578759,56045,42.882320,-107.620012,wyoming,2675
3127,331,578759,56017,42.882320,-107.620012,wyoming,2867


In [80]:
len(long_df.FIPS.unique())

3129

In [81]:
long_df.to_csv(save_path + f'COVID_{total_no_counties}_counties_long_pruned.csv')

# Merge with other variables

In [82]:
long_df.shape

(3129, 7)

In [83]:
daily.columns

Index(['FIPS', 'ID', 'County', 'Population', 'FIPSnew', 'County1', 'State',
       'Lat', 'Long_', 'density_sk', 'Date', 'cum_Confirmed',
       'daily_confirmed', 'cum_Vaccinated', 'daily_vaccinated',
       'per_daily_vaccinated', 'ContainmentHealthIndex',
       'ContainmentHealthIndex1', 'country', 'T'],
      dtype='object')

In [84]:
# drop original Population, Lat and Long_
daily.drop(['Population', 'Lat', 'Long_'], axis=1, inplace=True)

In [85]:
daily.shape, long_df.shape

((2124591, 17), (3129, 7))

In [86]:
all_long_df = long_df.merge(daily, on=['FIPS', 'ID', 'State'], how='inner')

In [87]:
all_long_df.shape

(2124591, 21)

In [88]:
all_long_df.head()

,ID_metacounty,Population,FIPS,Lat,Long_,State,ID,County,FIPSnew,County1,...,Date,cum_Confirmed,daily_confirmed,cum_Vaccinated,daily_vaccinated,per_daily_vaccinated,ContainmentHealthIndex,ContainmentHealthIndex1,country,T
0,0,619018,1089,34.444911,-85.998297,alabama,190,"madison county, alabama",1089,madison county,...,2020-01-22,0,0,0,0,0.0,0.0,0.0,US,0.420791
1,0,619018,1089,34.444911,-85.998297,alabama,190,"madison county, alabama",1089,madison county,...,2020-01-23,0,0,0,0,0.0,0.0,0.0,US,4.985396
2,0,619018,1089,34.444911,-85.998297,alabama,190,"madison county, alabama",1089,madison county,...,2020-01-24,0,0,0,0,0.0,0.0,0.0,US,8.402002
3,0,619018,1089,34.444911,-85.998297,alabama,190,"madison county, alabama",1089,madison county,...,2020-01-25,0,0,0,0,0.0,0.0,0.0,US,5.432438
4,0,619018,1089,34.444911,-85.998297,alabama,190,"madison county, alabama",1089,madison county,...,2020-01-26,0,0,0,0,0.0,0.0,0.0,US,3.982701


In [89]:
all_long_df.columns

Index(['ID_metacounty', 'Population', 'FIPS', 'Lat', 'Long_', 'State', 'ID',
       'County', 'FIPSnew', 'County1', 'density_sk', 'Date', 'cum_Confirmed',
       'daily_confirmed', 'cum_Vaccinated', 'daily_vaccinated',
       'per_daily_vaccinated', 'ContainmentHealthIndex',
       'ContainmentHealthIndex1', 'country', 'T'],
      dtype='object')

# Aggregate distance networks

## Flights

In [90]:
# merge with metacounty ID
flights_adj_mat = flights_adj_mat.merge(long_df[['ID', 'ID_metacounty']], on='ID').copy()
ID_col = flights_adj_mat['ID'].values
flights_adj_mat1 = flights_adj_mat.drop('ID', axis=1)
flights_adj_mat1.shape

(3129, 3130)

In [91]:
# group rows my metacounty ID and compute mean
flights_step1 = flights_adj_mat1.groupby('ID_metacounty', as_index=False).mean()

In [92]:
# transpose
flights_step1 = flights_step1.T.copy()
flights_step1.drop('ID_metacounty', inplace=True)
flights_step1.shape

(3129, 332)

In [93]:
# add ID column back
flights_step1['ID'] = ID_col

In [94]:
# merge with metacounty ID
flights_step1 = flights_step1.merge(long_df[['ID', 'ID_metacounty']], on='ID').copy()
flights_step1 = flights_step1.drop('ID', axis=1)

In [95]:
# group rows my metacounty ID and compute mean
flights_step2 = flights_step1.groupby('ID_metacounty', as_index=False).mean()
flights_step2 = flights_step2.T.copy()
flights_step2.drop('ID_metacounty', inplace=True)
flights_step2.shape

(332, 332)

In [96]:
# fill diagonal with zeros
flights_step2 = flights_step2.where(np.identity(flights_step2.shape[0]) != 1,0).copy()
flights_step2

,0,1,2,3,4,5,6,7,8,9,...,322,323,324,325,326,327,328,329,330,331
0,0.000000,0.000000,1.204956,0.000000,1.499261,0.466508,1.517879,0.023092,0.000000,90.684098,...,9.604288,0.0,1.302158,0.011912,0.043539,1.875833,1.883827,3.770333,46.060474,0.00000
1,0.000000,0.000000,1.498988,10.766026,0.000000,0.599722,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.298312,0.00000
2,1.204956,1.498988,0.000000,0.000000,0.115925,1.188628,0.000000,0.000000,0.000000,0.000000,...,0.312434,0.0,0.000000,0.025824,0.000000,0.158290,3.511468,0.000000,0.321847,0.25590
3,0.000000,10.766026,0.000000,0.000000,0.062934,0.265027,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
4,1.499261,0.000000,0.115925,0.062934,0.000000,0.044995,0.134993,0.002054,0.000000,8.064988,...,0.854157,0.0,0.115807,0.001059,0.003872,0.166827,0.167538,0.335314,4.103976,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
327,1.875833,0.000000,0.158290,0.000000,0.166827,0.063329,0.137133,37.746257,0.000000,5.343031,...,0.171720,0.0,135.764368,6.175954,12.172612,0.000000,224.781365,25.079919,159.967962,0.00000
328,1.883827,0.000000,3.511468,0.000000,0.167538,1.404885,0.000000,1013.906254,0.000000,175.934828,...,2.821335,0.0,3052.207380,146.025490,350.845014,224.781365,0.000000,510.368143,1768.672671,0.00000
329,3.770333,0.000000,0.000000,0.000000,0.335314,0.000000,0.199513,10.721841,0.000000,0.000000,...,0.097209,0.0,201.405760,1.873762,15.049398,25.079919,510.368143,0.000000,209.605434,0.00000
330,46.060474,1.298312,0.321847,0.000000,4.103976,0.176286,85.332435,123.378676,0.652208,507.482579,...,2.416581,0.0,2461.228114,32.942711,33.643024,159.967962,1768.672671,209.605434,0.000000,0.01556


In [97]:
flights_step2.to_csv(save_path + f'flights_{total_no_counties}_meta_capacity.csv')
#flights_step2.to_csv(save_path + f'flights_{total_no_counties}_meta_nocapacity.csv')

## Geodist

In [98]:
# merge with metacounty ID
geodist = geodist.merge(long_df[['ID', 'ID_metacounty']], on='ID').copy()
ID_col = geodist['ID'].values
geodist1 = geodist.drop('ID', axis=1)
geodist1.shape

(3129, 3130)

In [99]:
# group rows my metacounty ID and compute mean
geo_step1 = geodist1.groupby('ID_metacounty', as_index=False).mean()

In [100]:
# transpose
geo_step1 = geo_step1.T.copy()
geo_step1.drop('ID_metacounty', inplace=True)
geo_step1.shape

(3129, 332)

In [101]:
# add ID column back
geo_step1['ID'] = ID_col

In [102]:
# merge with metacounty ID
geo_step1 = geo_step1.merge(long_df[['ID', 'ID_metacounty']], on='ID').copy()
geo_step1 = geo_step1.drop('ID', axis=1)

In [103]:
# group rows my metacounty ID and compute mean
geo_step2 = geo_step1.groupby('ID_metacounty', as_index=False).mean()
geo_step2 = geo_step2.T.copy()
geo_step2.drop('ID_metacounty', inplace=True)
geo_step2.shape

(332, 332)

In [104]:
# fill diagonal with zeros
geo_step2 = geo_step2.where(np.identity(geo_step2.shape[0]) != 1,0).copy()
geo_step2

,0,1,2,3,4,5,6,7,8,9,...,322,323,324,325,326,327,328,329,330,331
0,0.000000e+00,1.366681e+05,3.133887e+05,3.847941e+05,1.482835e+05,1.958932e+05,5.217304e+06,2.254423e+06,2.396243e+06,2.403061e+06,...,7.665846e+05,5.881815e+05,9.961987e+05,1.072605e+06,1.288892e+06,1.045283e+06,9.603773e+05,1.213918e+06,9.474787e+05,2.061360e+06
1,1.366681e+05,0.000000e+00,2.277198e+05,2.580612e+05,1.025178e+05,1.175284e+05,5.257519e+06,2.182129e+06,2.337074e+06,2.328744e+06,...,8.961697e+05,7.173518e+05,1.081232e+06,1.153655e+06,1.366322e+06,1.140012e+06,1.056031e+06,1.306429e+06,1.037728e+06,2.059987e+06
2,3.133887e+05,2.277198e+05,0.000000e+00,1.809742e+05,3.128583e+05,1.835763e+05,5.476610e+06,2.300375e+06,2.475100e+06,2.442823e+06,...,1.003726e+06,8.291225e+05,1.299040e+06,1.372906e+06,1.586944e+06,1.352365e+06,1.267773e+06,1.520073e+06,1.252672e+06,2.260318e+06
3,3.847941e+05,2.580612e+05,1.809742e+05,0.000000e+00,3.176369e+05,2.214443e+05,5.420924e+06,2.147139e+06,2.330319e+06,2.287638e+06,...,1.130123e+06,9.522154e+05,1.315782e+06,1.383796e+06,1.591186e+06,1.382288e+06,1.299538e+06,1.545872e+06,1.276999e+06,2.168065e+06
4,1.482835e+05,1.025178e+05,3.128583e+05,3.176369e+05,0.000000e+00,1.877990e+05,5.174549e+06,2.127152e+06,2.275206e+06,2.274875e+06,...,8.872654e+05,7.100464e+05,1.008903e+06,1.079733e+06,1.290651e+06,1.071834e+06,9.886255e+05,1.236771e+06,9.678760e+05,1.980087e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
327,1.045283e+06,1.140012e+06,1.352365e+06,1.382288e+06,1.071834e+06,1.211182e+06,4.306586e+06,2.277421e+06,2.287832e+06,2.426849e+06,...,8.500572e+05,8.438916e+05,1.242679e+05,1.662044e+05,3.229979e+05,0.000000e+00,8.918011e+04,1.833067e+05,1.220584e+05,1.532297e+06
328,9.603773e+05,1.056031e+06,1.267773e+06,1.299538e+06,9.886255e+05,1.127165e+06,4.381833e+06,2.264691e+06,2.286173e+06,2.415251e+06,...,7.881383e+05,7.707937e+05,1.181773e+05,1.972188e+05,3.886009e+05,8.918011e+04,0.000000e+00,2.612042e+05,6.666998e+04,1.562229e+06
329,1.213918e+06,1.306429e+06,1.520073e+06,1.545872e+06,1.236771e+06,1.377734e+06,4.153014e+06,2.301469e+06,2.289841e+06,2.447998e+06,...,9.892710e+05,9.991613e+05,2.476014e+05,2.204424e+05,2.296811e+05,1.833067e+05,2.612042e+05,0.000000e+00,2.798602e+05,1.475298e+06
330,9.474787e+05,1.037728e+06,1.252672e+06,1.276999e+06,9.678760e+05,1.110040e+06,4.365319e+06,2.205032e+06,2.227394e+06,2.355580e+06,...,8.247077e+05,7.962257e+05,8.088767e+04,1.687160e+05,3.723538e+05,1.220584e+05,6.666998e+04,2.798602e+05,0.000000e+00,1.512422e+06


In [105]:
geo_step2.to_csv(save_path + f'geodist_{total_no_counties}_meta_pruned.csv')

## SCI

In [106]:
# merge with metacounty ID
sci = sci.merge(long_df[['ID', 'ID_metacounty']], on='ID').copy()
ID_col = sci['ID'].values
sci = sci.drop('ID', axis=1)
sci.shape

(3129, 3130)

In [107]:
# group rows my metacounty ID and compute mean
sci_step1 = sci.groupby('ID_metacounty', as_index=False).mean()

In [108]:
# transpose
sci_step1 = sci_step1.T.copy()
sci_step1.drop('ID_metacounty', inplace=True)
sci_step1.shape

(3129, 332)

In [109]:
# add ID column back
sci_step1['ID'] = ID_col

In [110]:
# merge with metacounty ID
sci_step1 = sci_step1.merge(long_df[['ID', 'ID_metacounty']], on='ID').copy()
sci_step1 = sci_step1.drop('ID', axis=1)

In [111]:
# group rows my metacounty ID and compute mean
sci_step2 = sci_step1.groupby('ID_metacounty', as_index=False).mean()
sci_step2 = sci_step2.T.copy()
sci_step2.drop('ID_metacounty', inplace=True)
sci_step2.shape

(332, 332)

In [112]:
# fill diagonal with zeros
sci_step2 = sci_step2.where(np.identity(sci_step2.shape[0]) != 1,0).copy()
sci_step2

,0,1,2,3,4,5,6,7,8,9,...,322,323,324,325,326,327,328,329,330,331
0,0.000000,73284.400000,27020.733333,33758.250000,137248.186667,118675.808696,2141.285714,1192.733333,1083.000000,1215.600000,...,1707.366667,2193.544000,913.600000,1010.687500,908.526316,828.425000,1139.800000,857.470000,1077.428571,1599.991304
1,73284.400000,0.000000,85252.466667,108973.500000,175693.400000,244054.304348,1776.071429,949.333333,997.428571,1443.000000,...,1129.533333,1494.840000,1800.000000,617.000000,597.947368,834.250000,4178.000000,601.300000,1334.142857,1498.913043
2,27020.733333,85252.466667,0.000000,335364.633333,29533.142222,179109.252174,2296.469048,1355.344444,1011.466667,1645.133333,...,1851.440000,1614.074667,1148.733333,898.458333,838.814035,808.058333,2872.733333,841.253333,1193.990476,1422.040580
3,33758.250000,108973.500000,335364.633333,0.000000,51957.425000,387523.032609,1909.700893,978.750000,1007.160714,1143.375000,...,1469.062500,1481.770000,913.625000,1105.820312,906.236842,1117.468750,2778.000000,896.031250,1246.857143,1766.380435
4,137248.186667,175693.400000,29533.142222,51957.425000,0.000000,186742.773913,1660.764286,1275.955556,1094.228571,1145.333333,...,1732.253333,2499.216000,839.400000,978.462500,846.800000,869.700000,1355.866667,789.930000,1276.371429,1857.049275
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
327,828.425000,834.250000,808.058333,1117.468750,869.700000,925.891304,2481.611607,1365.895833,1563.250000,1882.625000,...,814.991667,749.110000,82440.125000,96798.500000,30527.085526,0.000000,111392.000000,123248.187500,85689.928571,2303.652174
328,1139.800000,4178.000000,2872.733333,2778.000000,1355.866667,2677.826087,1459.785714,1799.833333,1214.428571,2556.000000,...,696.500000,656.080000,71207.000000,29458.875000,14494.105263,111392.000000,0.000000,38726.200000,119358.285714,1240.217391
329,857.470000,601.300000,841.253333,896.031250,789.930000,703.245652,3670.460714,1737.125000,2418.978571,1868.550000,...,929.501667,760.086000,55321.700000,69746.425000,125520.481579,123248.187500,38726.200000,0.000000,40625.714286,4131.486957
330,1077.428571,1334.142857,1193.990476,1246.857143,1276.371429,1277.229814,1939.040816,1657.642857,1628.775510,2170.285714,...,898.300000,722.645714,177161.142857,124248.660714,27727.962406,85689.928571,119358.285714,40625.714286,0.000000,2403.055901


In [113]:
sci_step2.shape

(332, 332)

In [114]:
sci_step2.to_csv(save_path + f'sci_index_{total_no_counties}_meta_pruned.csv')

# Long df with one row per FIPS but with aggregated data

In [115]:
ID_metacounty = all_long_df['ID_metacounty'].values
dates = all_long_df['Date'].values
# keep unique identifiers
df_keep = all_long_df[['ID_metacounty', 'ID', 'FIPS', 'Date', 'FIPSnew']]

# transform string vars
to_keep1 = ['ID_metacounty', 'State', 'country', 'County', 'County1', 'Date']
red_long_df1 = all_long_df[to_keep1]

# transform numerical vars
to_drop2 = ['County','State', 'country','County1','ID', 'FIPS', 'FIPSnew']
red_long_df2 = all_long_df.drop(to_drop2, axis=1)


In [116]:
%%time
# transform string vars
final_long_df1 = red_long_df1.groupby(['ID_metacounty', 'Date']).transform(lambda x: {"State":x.iloc[0],
                                                                              "country":x.iloc[0],
                                                                             "County":x.iloc[0],
                                                                         "County1":x.iloc[0]}[x.name]) 

final_long_df1['ID_metacounty_1'] = ID_metacounty
final_long_df1['Date_1'] = dates
final_long_df1.shape

CPU times: user 3min 43s, sys: 388 ms, total: 3min 43s
Wall time: 3min 43s


(2124591, 6)

In [117]:
%%time
# transform numerical vars
final_long_df2 = red_long_df2.groupby(['ID_metacounty', 'Date']).transform(lambda x: {"Population":x.iloc[0],
                                                                             "Lat":x.iloc[0], 
                                                                             "Long_":x.iloc[0],
                                                    "density_sk":x.mean(),
                                                        "cum_Confirmed":x.sum(),
                                                   "daily_confirmed":x.sum(),
                                                   "cum_Vaccinated":x.iloc[0], 
                                                   "daily_vaccinated":x.iloc[0], "per_daily_vaccinated":x.mean(), 
                                                   "ContainmentHealthIndex":x.iloc[0], 
                                                   "ContainmentHealthIndex1":x.iloc[0], 
                                                   "T":x.mean(),}[x.name])

final_long_df2['ID_metacounty_2'] = ID_metacounty
final_long_df2['Date_2'] = dates
final_long_df2.shape

CPU times: user 19min 57s, sys: 887 ms, total: 19min 58s
Wall time: 19min 56s


(2124591, 14)

In [118]:
# merge
final_long_df = pd.concat([final_long_df1.drop(['ID_metacounty_1', 'Date_1'], axis=1).copy(), 
                           final_long_df2.drop(['ID_metacounty_2', 'Date_2'], axis=1).copy(), 
                           df_keep], axis=1).copy()
final_long_df.shape

(2124591, 21)

In [119]:
final_long_df.to_csv(save_path + f'COVID_{total_no_counties}_counties_meta_long_pruned.csv')

In [120]:
final_long_df.columns

Index(['State', 'country', 'County', 'County1', 'Population', 'Lat', 'Long_',
       'density_sk', 'cum_Confirmed', 'daily_confirmed', 'cum_Vaccinated',
       'daily_vaccinated', 'per_daily_vaccinated', 'ContainmentHealthIndex',
       'ContainmentHealthIndex1', 'T', 'ID_metacounty', 'ID', 'FIPS', 'Date',
       'FIPSnew'],
      dtype='object')

# Aggregated df with one row per ID_metacounty-Date

In [121]:
to_drop = ['FIPS', 'ID', 'County',
       'FIPSnew', 'County1']
red_long_df = all_long_df.drop(to_drop, axis=1)
final_short_df = red_long_df.groupby(['ID_metacounty','Date'], as_index=False).agg({"Population":"first", 
                                                                                    "Lat":"first", "Long_":"first",
                                                                                    "State":"first",
                                                    "density_sk":"mean", "Date":"first",
                                                        "cum_Confirmed":"sum",
                                                   "daily_confirmed":"sum",
                                                   "cum_Vaccinated":"first", 
                                                   "daily_vaccinated":"first", "per_daily_vaccinated":"mean", 
                                                   "ContainmentHealthIndex":"first", 
                                                   "ContainmentHealthIndex1":"first", "country":"first",
                                                   "T":"mean",})
final_short_df.shape

(225428, 16)

In [122]:
final_short_df.to_csv(save_path + f'COVID_{total_no_counties}_counties_meta_agg_pruned.csv')

In [123]:
final_short_df.columns

Index(['ID_metacounty', 'Population', 'Lat', 'Long_', 'State', 'density_sk',
       'Date', 'cum_Confirmed', 'daily_confirmed', 'cum_Vaccinated',
       'daily_vaccinated', 'per_daily_vaccinated', 'ContainmentHealthIndex',
       'ContainmentHealthIndex1', 'country', 'T'],
      dtype='object')